<a href="https://colab.research.google.com/github/PeterNaggschga/Letter-Variations-in-First-Names-IS/blob/main/LetterVariationsFirstNames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RadixTree
Implementation eines RadixTrees.

In [ ]:
class RadixTree:
    """contains a dict, which links to multiple RadixTree's. The key to another tree is a str and
    if a transition ends in a word, the isWord-variable is True
    """

    def __init__(self, isWord=False, transitions=None):
        if transitions is None:
            transitions = dict()
        self.isWord = isWord
        self.transitions = transitions

    def insertWord(self, word):
        """inserts a word into the tree

        Args:
            word (str): is the str, that gets inserted
        """
        for i in range(0, len(word)):
            # goes through 'tobi' with 'tobi', 'tob', 'to', 't'
            possibleTransition = word[:len(word) - i]
            if self.transitions.get(possibleTransition) is not None:
                child = self.transitions.get(possibleTransition)
                if possibleTransition == word:
                    child.isWord = True
                else:
                    child.insertWord(word[len(possibleTransition):])
                return

            for key in self.transitions.keys():
                if possibleTransition == key[:len(possibleTransition)]:
                    child = self.transitions.pop(key)
                    newDict = dict()
                    newDict[key[len(possibleTransition):]] = child
                    self.transitions[possibleTransition] = RadixTree(possibleTransition == word, newDict)
                    if possibleTransition != word:
                        self.transitions[possibleTransition].insertWord(word[len(possibleTransition):])
                    return

        self.transitions[word] = RadixTree(True, dict())

    def __strRecursive__(self, timesOfIndentation, lengthOfTransitionString):
        result = ""
        if self.isWord:
            result += "."
        keys = list(self.transitions.keys())
        keys.sort()
        for key in keys:
            recursiveResult = self.transitions[key].__strRecursive__(timesOfIndentation + lengthOfTransitionString,
                                                                     len(key))
            result += "\n" + (timesOfIndentation + lengthOfTransitionString) * "_" + key + recursiveResult
        return result

    def __str__(self):
        """generates a readable str, containing all class variables (the tree).

        Returns:
            str: the generated str
        """
        return self.__strRecursive__(0, 0)

    def getSimilarWordsOfSameLength(self, maximumDifferentLetters, word):
        """compares the given word with entries of the same length. Returns all of them with less or equal different letters than with maximumDifferentLetters described

        Args:
            maximumDifferentLetters (int): limits the amount of accepted different letters when comparing 2 words
            word (str): the given word

        Returns:
            list: returns a list of similar words with the same length. Does contain itself
        """
        if word == "":
            return [word] if self.isWord else []

        resultList = list()
        for key in self.transitions.keys():
            if len(key) > len(word):
                continue
            differences = 0
            for i in range(0, len(key)):
                differences += word[i] != key[i]
                if differences > maximumDifferentLetters:
                    break

            if differences > maximumDifferentLetters:
                continue
            resultTmp = self.transitions[key].getSimilarWordsOfSameLength(maximumDifferentLetters - differences,
                                                                          word[len(key):])
            if resultTmp:
                resultList.extend([key + tmp for tmp in resultTmp])
        return resultList


# RadixTreesByWordLength
Ein Container, der Wörter je nach Länge in unterschiedliche RadixTrees ablegt und ansonsten wie ein Tree agiert.

In [ ]:
class RadixTreesByWordLength:
    """contains a dict, which links to multiple RadixTree's, each storing words of the same length
    """

    def __init__(self):
        self.radixTrees = dict()

    def insertWord(self, word):
        """inserts a word into the tree

        Args:
            word (str): is the str, that gets inserted
        """
        length = len(word)
        if self.radixTrees.get(length) is None:
            self.radixTrees[length] = RadixTree()
        self.radixTrees[length].insertWord(word)

    def __str__(self):
        """generates a readable str, containing all class variables (the tree)

        Returns:
            str: the generated str
        """
        tmp = ""
        lengthStr = self.radixTrees.keys()
        sortedLengths = [int(lengths) for lengths in lengthStr]
        sortedLengths.sort()

        for length in sortedLengths:
            tmp += "RadixTree with words of length " + str(length) + ":"
            tmp += self.radixTrees[length].__str__()
            tmp += "\n"
        return tmp

    def getSimilarWordsOfSameLength(self, maximumDifferentLetters, word):
        """compares the given word with entries of the same length. Returns all of them with less or equal different letters than with maximumDifferentLetters described

        Args:
            maximumDifferentLetters (int): limits the amount of accepted different letters when comparing 2 words
            word (str): the given word

        Returns:
            list: returns a list of similar words with the same length. Does contain itself
        """
        length = len(word)
        if self.radixTrees.get(length) is None:
            return []
        return self.radixTrees[length].getSimilarWordsOfSameLength(maximumDifferentLetters, word)


# Name Extraction

In [ ]:
# install and import Entrez and Medline first
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    from Bio import Entrez, Medline
except:
    # One of these 2 lines should work
    # !pip install Bio
    install('Bio')
from Bio import Entrez, Medline

def createEmptyFile(nameOfFile):
    with open(nameOfFile, "w+") as myfile:    # create file which stores the names
        try:
            myfile.write("")
        finally:
            myfile.close()

def appendNamesToFile(filename, names):
    with open(filename, "a") as myfile:   #fill file
        try:
            tmp = ""
            for name in names:
                name = name.lower()
                tmp += name + " "
            myfile.write(tmp)
        finally:
            myfile.close()

def getPapers(myQuery, maxPapers, myEmail="freytag64@gmail.com"):
    """retrieves some Papers from Pubmed

    Args:
        myQuery (str): is the given Query 
        maxPapers (int): is a limit of the number of papers, which will be retrieved
        myEmail (str, optional): an email. Defaults to "freytag64@gmail.com".

    Returns:
        list: papers as list of dictionarys containing abstract, authors, ...
    """
    # Get articles from PubMed
    Entrez.email = myEmail
    record = Entrez.read(Entrez.esearch(db="pubmed", term=myQuery, retmax=maxPapers))
    idlist = record["IdList"]
    print("\nThere are %d records for %s." % (len(idlist), myQuery.strip()))
    records = Medline.parse(Entrez.efetch(db="pubmed", id=idlist, rettype="medline", retmode="text"))
    return list(records)


def retrieveAllFirstNames(records):
    """takes list of papers (each is a dict) and extracts the first names as a combined list with a regular expression 

    Args:
        records (list): list of papers

    Returns:
        list: list of first names as str's
    """
    # retrieves first names from authors with regular expressions
    # FAU - Full Author Name
    firstNameList = list()
    for record in filter(lambda x: 'FAU' in x, records):
      #First check if FAU is empty/None then continue with next Author
        if(record['FAU'] == None):
            continue
        
        #iterate over all Full names and get valif first names
        for fullName in record['FAU']:
            # since names are formatted like 'Lastname, Firstnames', split by ',' to get Firstnames
            firstName = fullName.split(',')

            #only include first names >= 2 to exclude shortcuts
            if len(firstName) >= 2:
                firstName = firstName[1].strip()
                #iterate over all First names 
                names1 = list(filter(lambda x: len(x) > 1, firstName.split()))    #removes P or l
                for name in names1:
                    if len(name) == 2 and name[1] == ".":   #check for firstname shortcuts P. or l. and remove them
                        names1.remove(name)
                    if "-" in name and name in names1:  #check for firstname shortcuts P-G or l-r and remove them
                        names1.remove(name)             #remove all double First names with '-' in between
                    if any(char.isdigit() for char in name) and name in names1:   #remove all strings with numbers. For example 2nd or 3rd
                        names1.remove(name)
                
                names1 = list(map(lambda x: x.lower(), names1))   #all names are written in lower cases
                firstNameList.extend(names1)

    return firstNameList

In [ ]:
if __name__ == "__main__":
    maxPapers = 10000 
    names = list()

    createEmptyFile("2022FirstNames")

    for day in range(1, 32):
        try:
            #retrieve names of January 2022
            myQuery = "(\"2022/01/" + str(day) + "\"[Date - Publication] : \"2022/01/" + str(day) + "\"[Date - Publication])"
            records = getPapers(myQuery, maxPapers)
            firstnames = retrieveAllFirstNames(records)
            names.extend(firstnames)

            appendNamesToFile("2022FirstNames", names)

            print(str(len(firstnames)) + " new names read in")
        except Exception as e: 
            print(str(e))  
        pass

#Clustering
Clustern der Namen mit einer DBSCAN-ähnlichen Methode.

Unterschiede:
- zu große Cluster werden strenger erneut geclustert (maximale Größe: 1000 Namen)
- beim Prüfen, ob ein Knoten ein Kernknoten ist, zählen bereits erkundete Knoten nicht mehr
- Namen mit Länge 2 oder kürzer werden ignoriert

Ein Kernknoten braucht mindestens 6 unbesuchte Nachbarn um als Kernknoten zu gelten.

Eliminiert nicht erreichbare Knoten.

Knoten: einzelne Namen

Knotenübergänge (ungerichtet) zwischen Namen gleicher Länge mit maximaler Hamming-Distanz 1

In [ ]:
def Dbscan(names : list, min_samples : int, min_word_length : int, max_cluster_length : int) -> list:
    """clusters list of names with DBSCAN

    Args:
        names (list): list of names
        min_samples (int): the number of samples in a neighborhood for a point to be considered a core point
        min_word_length (int): only words with this length or longer are clustered
        max_cluster_length (int): maximum size of cluster. If found cluster are bigger, they will devided
    Returns:
        list: list of clusters (each cluster is a list of names)
    """
    
    tree = RadixTreesByWordLength()
    clusters = list()
    names = list(set(names)) #remove duplicates
    names = list(filter(lambda x : len(x) >= min_word_length, names))
    names.sort(reverse=True)
    
    tmp = list()
    for n in names:
        tmp.append(n.lower())
    names = tmp
    
    #construct the RadixTree
    for n in names:
        tree.insertWord(n)

    
    #clustering
    while len(names) > 0:
        name = names.pop()
        cluster = set()
        stack = [name]
        
        #fill cluster
        while len(stack) > 0:
            node = stack.pop()
            
            #check if name is a core-sample
            neighbors = tree.getSimilarWordsOfSameLength(1, node)
            #already found nodes are not considered as neighbors
            toBeRemoved = list()
            for n in neighbors:
                if n in cluster:
                    toBeRemoved.append(n)
                    continue
                for c in clusters:
                    if n in c:
                        toBeRemoved.append(n)
            for n in toBeRemoved:
                neighbors.remove(n) 
                
            #add new names to cluster
            if len(neighbors) >= min_samples:
                for n in filter(lambda x : x in names, neighbors):
                    names.remove(n)
                stack.extend(neighbors)
                cluster.update(set(neighbors))
        
        cluster = list(cluster)
        #add filled cluster
        if len(cluster) > 1 and len(cluster) <= max_cluster_length:
            clusters.append(cluster)
        #if a cluster is too big, cluster it again
        elif len(cluster) > max_cluster_length:
            smallerClusters = Dbscan(cluster, min_samples + 5, min_word_length, max_cluster_length)
            clusters.extend(smallerClusters)    

    return clusters

def saveClustersInFile(filename, clusters):
    with open(filename, "w+") as myfile:
        try:
            tmp = ""
            for cluster in clusters:
                for name in cluster:
                    tmp += name + " "
                tmp += "\n"
            myfile.write(tmp)
        finally:
            myfile.close()        

In [ ]:
print("clustering (takes a while):")
clusters = Dbscan(names, 3, 3, 600)

print(str(len(clusters)) + " different clusters")
i = 0
for c in clusters:
    i += len(c)
print(str(i) + " names clustered")

#save clusters in file
saveClustersInFile("clusteredNames", clusters)

# SubstitutionMatrix
Implementation einer Substitutionsmatrix, angelehnt an BLOSUM. Ignoriert aktuell alle Buchstaben, die nicht im ASCII kodiert sind.

Matrixeinträge werden nach folgender Formel berechnet: 
$$ S_{i,j} = \frac{1}{\lambda_2} \log_2{(\lambda_1 \cdot \frac{p_{ij}}{q_i \cdot q_j})} $$

$ p_{ij} $ ... Wahrscheinlichkeit, dass (innerhalb eines Clusters) i durch j ersetzt wird \\
$ q_i, q_j $ ... Wahrscheinlichkeiten, dass i bzw. j in einem Wort auftritt \\
$ \lambda_i $ ... (frei wählbare) Skalierungsfaktoren \\
Dabei wird $ \lambda_1 $ genutzt, um die Werte in der Matrix in positive oder negative Richtung zu verschieben. 
$ \lambda_2 $ skaliert größe der Werte linear. 
Die Funktion SubstitutionMatrix.autoscale() skaliert die Werte der Matrix standardmäßig so, dass für alle Werte gilt $ S_{i,j} \in [-5, 5] $ (Abweichungen von bis zu 0.1% möglich).

---

Die Substitutionswahrscheinlichkeit $p_{ij}$ wird iterativ für jedes Cluster $C$ berechnet. Dabei werden alle Wörter des Clusters zeichenweise an den Stellen $k=1,...,l_C$ betrachtet, wobei $l_C$ die Länge der Wörter im Cluster $C$ angibt. Nun wird für jedes Buchstabenpaar $(i, j)$ die Wahrscheinlichkeit des gemeinsamen Auftretens in $C$ an Stelle $k$ wie folgt berechnet:

$$ p_{ij, k}^{(C)} = \frac{f_k^{(C)}(i, j)}{\displaystyle \sum_{a = 1} ^{n} \sum_{b = a} ^{n} f_k^{(C)}(a, b)} $$

$n$ ... Anzahl Buchstaben im betrachteten Alphabet (bei uns gilt n = 26)

Dabei gilt für die Anzahl möglicher Buchstabenpaare $(i, j)$ in Cluster $C$ an Stelle $k$:

Für $i \neq j$ gilt: 
$$ f_k^{(C)}(i, j) = o_{k, i}^{(C)} \cdot o_{k, j}^{(C)} $$

sonst: 
$$ f_k^{(C)}(i, i) = \frac{o_{k, i}^{(C)} \cdot (o_{k, i}^{(C)} - 1)}{2} $$

wobei $o_{k, i}^{(C)}$ angibt, wie oft im Cluster $C$ an Stelle $k$ der Buchstabe an Stelle $i$ des Alphabets vorkommt.

---

Die so erhaltenen Substitutionswahrscheinlichkeit $p_{ij, k}^{(C)}$ werden nun aufaddiert. Dabei wird nach der Anzahl der entsprechenden Substitutionen $o_{k, i}^{(C)}$ gewichtet:

$$ p_{ij} = \frac{1}{o_{ij}} \displaystyle \sum_{C \in M} \sum_{k=1}^n ((o_{k, i}^{(C)} + o_{k, j}^{(C)}) \cdot p_{ij, k}^{(C)} ) $$

wobei $M$ die Menge aller Cluster und $o_{ij}$ die Gesamtzahl der gemeinsamen Vorkommen von $i$ und $j$ darstellt.

---

https://bioinformaticshome.com/bioinformatics_tutorials/sequence_alignment/substitution_matrices.html

In [ ]:
import numpy as np
from collections import defaultdict


class SubstitutionMatrix:

    def __init__(self, names: list, clusters: list, inner_scaling: float = 10, outer_scaling: float = 1):
        """
        Initializes and calculates a Substitution-Matrix out of the given names, clustering and scaling

        :param names: list of all names used to calculate matrix
        :param clusters: list of clusters which are lists of similar names
        :param scaling: optional scaling factor
        """
        self.letters = [
            'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']

        self.name_occurrences = self.calc_name_occurrences(names)

        self.letter_prob = self.calc_letter_prob()

        self.clusters = clusters

        self.inner_scaling = inner_scaling

        self.outer_scaling = outer_scaling

        self.sub_probs = self.calc_sub_probs()

        print("Calculating substitution matrix...")
        self.substitution_matrix = self.calc_substitution_matrix()
        print("done")

    def calc_name_occurrences(self, all_names: list) -> dict:
        """
        Calculates the number of duplicates for each name in all_names

        :param all_names: list of all names used to calculate the matrix
        :return: a dictionary mapping every name to its number of occurrences
        """
        print("Calculating name occurrences...")

        result = dict()
        names, count = np.unique(all_names, return_counts=True)
        for i in range(len(names)):
            result[names[i]] = count[i]

        print("done")
        return result

    def calc_letter_prob(self) -> dict:
        """
        Calculates the probability of every letter to be in a name

        :return: a dictionary mapping every letter to its probability of being in a name
        """
        print("Calculating probability of letters...")

        letter_counts = defaultdict(int)

        n = 0
        for (name, count) in self.name_occurrences.items():
            for x in {c for c in name}:
                if x in self.letters:
                    letter_counts[x] += count
            n += count

        letter_prob = defaultdict(int)
        for (x, occurrences) in letter_counts.items():
            letter_prob[x] = occurrences / n

        print("done")
        return letter_prob

    def calc_sub_probs(self) -> dict:
        """
        Calculates the probability of one letter to be substituted by another letter

        :return: a matrix with substitution probabilities for each letter
        """
        print("Calculating probability of substitutions...")

        sub_probs = dict()
        for x in self.letters:
            sub_probs[x] = dict()
            for y in self.letters:
                sub_probs[x][y] = (0, 0)  # (prob, #examples)

        for cluster in self.clusters:
            for i in range(len(cluster[0])):
                occurrences = defaultdict(int)
                for name in cluster:
                    n = self.name_occurrences[name]
                    letter = name[i]
                    if letter in self.letters:
                        occurrences[letter] += n

                f = lambda x, y: (occurrences[x] * (occurrences[x] - 1) / 2) if x == y else occurrences[x] * occurrences[y]
                keys = list(occurrences.keys())
                for a in keys:
                    for b in keys:
                        (p_old, n_old) = sub_probs[a][b]
                        fab = f(a, b)
                        if fab:
                            p_new = fab / sum([f(x, y) for x in keys for y in keys[keys.index(x):]])
                        else:
                            p_new = 0
                        
                        n_new = occurrences[a] + occurrences[b]
                        sub_probs[a][b] = ((1 / (n_old + n_new)) * (n_old * p_old + n_new * p_new), n_old + n_new)

        for a in self.letters:
            for b in self.letters:
                sub_probs[a][b] = sub_probs[a][b][0]
        print("done")
        return sub_probs

    def calc_substitution_matrix(self) -> dict:
        """
        Calculates the Substitution-Matrix

        :return: a matrix with substitution values for each letter
        """
        matrix = defaultdict(dict)
        min = float("inf")
        for a in self.letters:
            for b in self.letters:
                try:
                    if self.sub_probs[a][b] == 0:
                        matrix[a][b] = "min"
                    else:
                        matrix[a][b] = (1 / self.outer_scaling) * np.log(self.inner_scaling * self.sub_probs[a][b] / (self.letter_prob[a] * self.letter_prob[b]))
                        if matrix[a][b] < min:
                            min = matrix[a][b]
                            # print((a, b))
                except ZeroDivisionError:
                    matrix[a][b] = "min"

        for a in matrix.keys():
            for b in matrix[a].keys():
                if matrix[a][b] == "min":
                    # print((a, b))
                    matrix[a][b] = min

        return matrix

    def autoscale(self, exp_bounds: float = 5, derivation: float = 0.001):
        print(f"Scaling automatically for bounds {-exp_bounds} +/- {derivation} and {exp_bounds} +/- {derivation}...")
        (min, max) = self.get_bounds()
        print(f"Old scalings (l1, l2) = {(self.inner_scaling, self.outer_scaling)}")
        
        correction = self.inner_scaling / 2
        direction = 1 if abs(min) > abs(min) else -1
        while abs(abs(max) - abs(min)) > derivation:
            new_scaling = self.inner_scaling + direction * correction
            if new_scaling <= 0:
                new_scaling = 1
            self.set_scaling(inner_scaling=new_scaling)

            # print((new_scaling, min, max))
            (min, max) = self.get_bounds()
            direction_new = 1 if abs(min) > abs(max) else -1
            if direction_new != direction:
                correction /= 2
                direction = direction_new
        
        (min, max) = self.get_bounds()
        outer_scaling = (max + abs(min)) / (2 * exp_bounds)
        self.set_scaling(self.inner_scaling, outer_scaling)
        print(f"(min, max) = {self.get_bounds()}")

        print(f"New scalings (l1, l2) = {(self.inner_scaling, self.outer_scaling)}")
        print("done")

    def get_bounds(self):
        """
        Returns a triple of values representing minimum, maximum and average of the substitution matrix

        :return:
        """
        vals = [self.substitution_matrix[a][b] for a in self.letters for b in self.letters]
        return (np.min(vals), np.max(vals))

    def set_names(self, names: list, clusters: list = None):
        """
        Sets a new list of names (and a new clustering) as basis and calculates the new Substitution-Matrix

        :param names: list(str)
        :param clusters: list(list(str))
        :return:
        """
        self.__init__(names, clusters if clusters else self.clusters, self.scaling)

    def set_clusters(self, clusters: list):
        """
        Sets a new clustering and calculates the new Substitution-Matrix

        :param clusters: list(list(str))
        :return:
        """
        self.clusters = clusters
        self.sub_probs = self.calc_sub_probs()
        self.substitution_matrix = self.calc_substitution_matrix()

    def set_scaling(self, inner_scaling: float = 1, outer_scaling: float = 1):
        """
        Set a new scaling and calculate the new Substitution-Matrix
        :param scaling:
        :return:
        """
        if self.inner_scaling != inner_scaling or self.outer_scaling != outer_scaling:
            self.inner_scaling = inner_scaling
            self.outer_scaling = outer_scaling
            self.substitution_matrix = self.calc_substitution_matrix()

    def __str__(self):
        tmp = "Substitutionsmatrix:\n  "
        for to in self.letters:
            tmp += "\t" + to
        tmp += "\n"

        for fr in self.letters:
            tmp += fr
            for to in self.letters:
                strEntry = str(self.substitution_matrix[fr][to])
                if len(strEntry) < 4:
                    strEntry = (4 - len(strEntry)) * "0" + strEntry
                tmp += "\t" + strEntry[:4]
            tmp += "\n"
        return tmp


In [ ]:
names = [("klara", 2), ("clara", 4), ("clars", 4), ("klaus", 10), ("claus", 5), ("klaas", 15)]
all_names = []
for (name, count) in names:
    for i in range(count):
        all_names.append(name)

clusters = [["klaus", "claus", "klaas"], ["klara", "clara", "clars"]]

matrix = SubstitutionMatrix(all_names, clusters)
matrix.autoscale()
print(matrix)

Calculating name occurrences...
done
Calculating probability of letters...
done
Calculating probability of substitutions...
done
Calculating substitution matrix...
done
Scaling automatically for bounds -5 +/- 0.001 and 5 +/- 0.001...
Old scalings (l1, l2) = (10, 1)


KeyboardInterrupt: ignored

# Kleine Demo

In [ ]:
maxPapers = 10000  # limit the number of papers retrieved
myQuery = "(\"2021/01/20\"[Date - Publication] : \"2021/01/20\"[Date - Publication])"
records = getPapers(myQuery, maxPapers)


There are 4928 records for ("2021/01/20"[Date - Publication] : "2021/01/20"[Date - Publication]).


In [ ]:
names = retrieveAllFirstNames(records)

print(list(np.unique(names)))

['aa', 'aaditya', 'aafke', 'aakanksha', 'aaliyah', 'aalt', 'aamer', 'aamir', 'aanchal', 'aarent', 'aarno', 'aaron', 'aarooran', 'aarti', 'aasems', 'aayush', 'abani', 'abate', 'abay', 'abbas', 'abbie', 'abby', 'abd', 'abdalazeem', 'abdalghani', 'abdallah', 'abdel', 'abdelahhad', 'abdelaziz', 'abdelhafid', 'abdelhamid', 'abdelkader', 'abdellatif', 'abdelmoumen', 'abdelouahed', 'abdelsattar', 'abdelwahab', 'abdenacer', 'abdenbi', 'abderrahim', 'abderraouf', 'abdessamad', 'abdolreza', 'abdou', 'abdoulaye', 'abdul', 'abdulaziz', 'abdulhak', 'abdulkadir', 'abdulkarim', 'abdullah', 'abdullatif', 'abdulmajeed', 'abdulmunem', 'abdulrahman', 'abdulwahed', 'abdurrahman', 'abdurrashid', 'abebaw', 'abelardo', 'abhas', 'abhay', 'abhijit', 'abhik', 'abhilash', 'abhilasha', 'abhinav', 'abhiram', 'abhisek', 'abhishek', 'abi', 'abid', 'abigail', 'abimael', 'abina', 'abinaya', 'abiodun', 'abir', 'abolfazl', 'abou', 'aboubakr', 'abraham', 'abrahao', 'abram', 'abreu', 'abri', 'abu', 'abuduxikuer', 'abukar'

In [ ]:
clusters = Dbscan(names, 3, 4, 600)
for cluster in clusters:
    print(cluster)


['sarat', 'maram', 'samir', 'sahar', 'semah', 'samer', 'murat', 'samar', 'farah', 'aamir', 'aamer', 'saber', 'marat', 'sarit', 'sagar', 'tamer', 'salar', 'zarah', 'salah', 'yarah', 'sameh', 'sadat', 'maral', 'sabah', 'samah', 'sarah', 'tamar', 'simar']
['aaron', 'baron', 'arron', 'yaron']
['avani', 'amani', 'abani']
['abby', 'abay', 'aday', 'ajay']
['abhas', 'abhay', 'abbas']
['anina', 'erina', 'taiga', 'alina', 'aline', 'amira', 'irina', 'taina', 'amina', 'eline', 'iryna', 'alice', 'trina', 'elona', 'tiina', 'elina', 'amine', 'aliaa', 'trinh', 'taira', 'amida', 'elana', 'zaina', 'alisa', 'trine', 'elisa', 'erika', 'abina', 'taija', 'erica', 'irena', 'adina', 'irisa', 'erinn', 'jaina', 'tayna', 'amika', 'elena']
['bana', 'rong', 'rima', 'maru', 'ning', 'cong', 'lana', 'gong', 'lene', 'ping', 'leng', 'hong', 'risa', 'kong', 'aini', 'line', 'xana', 'kang', 'jana', 'rika', 'jang', 'seng', 'riya', 'keng', 'mars', 'lins', 'rino', 'bara', 'lena', 'long', 'deng', 'lata', 'sang', 'mara', 'qina

In [ ]:
matrix = SubstitutionMatrix(names, clusters)
matrix.autoscale()
#matrix.set_scaling(inner_scaling=1)

print(matrix)

Calculating name occurrences...
done
Calculating probability of letters...
done
Calculating probability of substitutions...
done
Calculating substitution matrix...
done
Scaling automatically for bounds -5 +/- 0.001 and 5 +/- 0.001...
Old scalings (l1, l2) = (10, 1)
(min, max) = (-4.999946988538749, 5.000053011461252)
New scalings (l1, l2) = (8.2958984375, 1.603250174539547)
done
Substitutionsmatrix:
  	a	b	c	d	e	f	g	h	i	j	k	l	m	n	o	p	q	r	s	t	u	v	w	x	y	z
a	1.69	1.28	0.11	-0.7	1.09	0.91	1.96	0.83	0.62	0.93	0.82	0.48	0.05	0.56	1.18	0.29	0.87	-0.1	0.53	0.28	1.14	-0.7	2.52	-0.0	0.91	-0.1
b	1.28	4.28	0.56	1.54	1.90	2.28	2.53	2.16	1.75	2.46	2.43	1.67	2.01	1.19	2.29	2.36	3.28	1.98	1.12	2.34	2.70	3.14	2.51	3.18	1.73	2.06
c	0.11	0.56	3.62	1.43	0.12	1.20	1.48	1.80	1.49	2.07	2.91	0.68	1.41	1.63	0.96	1.64	1.87	1.88	2.16	1.55	-2.2	1.57	2.05	2.13	0.16	3.71
d	-0.7	1.54	1.43	3.54	0.31	0.87	1.10	1.92	0.15	1.53	2.52	1.50	1.79	1.11	1.97	1.18	2.35	1.13	1.38	1.72	-4.9	2.09	1.41	2.09	1.79	2.79
e	1.09	1.90	0.

# Große Demo

Es werden Namen aus dem Januar 2022 gesammelt, geclustert und daraus die Matrix gebildet. 

Die Zwischenergebnisse werden dazu in den Dateien "2022FirstNames" und "clusteredNames" abgelegt und können dort eingesehen werden. Das Endergebnis wird in "finalMatrix" gespeichert.

In [ ]:
names = list()
createEmptyFile("2022FirstNames")

for day in range(1, 32):
    try:
        #retrieve names of January 2022
        myQuery = "(\"2022/01/" + str(day) + "\"[Date - Publication] : \"2022/01/" + str(day) + "\"[Date - Publication])"
        records = getPapers(myQuery, 10000)
        firstnames = retrieveAllFirstNames(records)
        names.extend(firstnames)

        #fill file with names
        appendNamesToFile("2022FirstNames", firstnames)

        print(str(len(firstnames)) + " new names read in")
    except Exception as e: 
        print(str(e))  
        
print(str(len(list(np.unique(names)))) + " unique names in total")


There are 9999 records for ("2022/01/1"[Date - Publication] : "2022/01/1"[Date - Publication]).
64796 new names read in

There are 985 records for ("2022/01/2"[Date - Publication] : "2022/01/2"[Date - Publication]).
5890 new names read in

There are 3587 records for ("2022/01/3"[Date - Publication] : "2022/01/3"[Date - Publication]).
24256 new names read in

There are 4859 records for ("2022/01/4"[Date - Publication] : "2022/01/4"[Date - Publication]).
32820 new names read in

There are 5360 records for ("2022/01/5"[Date - Publication] : "2022/01/5"[Date - Publication]).
35829 new names read in

There are 4390 records for ("2022/01/6"[Date - Publication] : "2022/01/6"[Date - Publication]).
29350 new names read in

There are 5192 records for ("2022/01/7"[Date - Publication] : "2022/01/7"[Date - Publication]).
35083 new names read in

There are 1754 records for ("2022/01/8"[Date - Publication] : "2022/01/8"[Date - Publication]).
11125 new names read in

There are 872 records for ("2022/

In [ ]:
print("clustering (takes a while):")
clusters = Dbscan(names, 3, 4, 600)

print(str(len(clusters)) + " different clusters")
i = 0
for c in clusters:
    i += len(c)
print(str(i) + " names clustered")

#save clusters in file
saveClustersInFile("clusteredNames", clusters)

clustering (takes a while):
5227 different clusters
42412 names clustered


In [ ]:
matrix = SubstitutionMatrix(names, clusters)
#matrix.autoscale()
matrix.set_scaling(inner_scaling=1)

print(matrix)

#save matrix in file
with open("finalMatrix", "w+") as myfile:
    try:
        myfile.write(str(matrix))
    finally:
        myfile.close()

Calculating name occurrences...
done
Calculating probability of letters...
done
Calculating probability of substitutions...
done
Calculating substitution matrix...
done
Substitutionsmatrix:
  	a	b	c	d	e	f	g	h	i	j	k	l	m	n	o	p	q	r	s	t	u	v	w	x	y	z
a	0.68	-0.7	-1.4	-2.0	-0.7	-0.8	0.60	-1.7	-1.2	-1.1	-0.4	-2.0	-2.1	-1.9	-0.8	-0.4	0.61	-2.0	-1.7	-1.6	-0.7	-0.4	1.00	-1.5	-0.9	-1.1
b	-0.7	4.87	0.68	1.31	-0.6	1.30	0.91	0.56	-1.2	1.37	1.69	0.60	0.47	-0.8	-1.1	1.18	1.94	-0.0	0.27	0.58	-0.4	2.66	2.04	1.96	0.49	1.70
c	-1.4	0.68	3.90	0.27	-1.3	-0.2	-0.0	-0.3	-0.6	0.07	2.20	-1.1	0.17	-1.1	-0.2	0.77	0.26	-1.2	1.04	0.59	-0.9	0.96	1.19	1.13	-0.4	3.08
d	-2.0	1.31	0.27	3.65	-1.2	0.68	-0.2	0.09	-2.0	0.72	0.84	-0.1	-0.3	-0.8	-1.0	-0.1	1.21	-0.0	-0.3	0.47	-0.0	1.22	0.66	0.36	0.87	1.36
e	-0.7	-0.6	-1.3	-1.2	1.58	-0.0	-0.6	-1.0	-0.8	-0.6	-0.6	-0.8	-2.2	-1.8	-0.4	-0.5	-0.0	-1.9	-1.1	-1.0	-0.2	-0.8	-0.7	-0.3	-0.5	-0.0
f	-0.8	1.30	-0.2	0.68	-0.0	5.42	1.10	0.53	-2.4	0.50	1.03	0.35	0.99	-1.2	-0.1	3.07	1.63	-0.3	0.7